# 🏇 JRA 詳細情報スクレイピング (Stage 2/2)

このノートブックは**馬履歴+血統情報**を取得します。

## 📊 前提条件
- `Colab_JRA_Basic.ipynb` を実行済み
- `database_basic.csv` が存在する

## 📊 取得データ
- 馬履歴ページから65カラム(past_1_* ~ past_5_*)
- 血統ページから3カラム(father, mother, bms)

## 🚀 次のステップ
`merge_jra_data.py` でデータを結合 → `database.csv`

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 🛡️ Keep-Alive (アイドルタイムアウト回避)
from IPython.display import display, Javascript

display(Javascript('''
function ClickConnect(){
    console.log("Keep-alive: Working");
    var buttons = document.querySelectorAll("colab-connect-button");
    buttons.forEach(function(btn){
        btn.click();
    });
}
setInterval(ClickConnect, 60000);
console.log("Keep-alive script started - clicks every 60 seconds");
'''))

print("✅ Keep-alive activated (auto-clicks every 60 seconds)")
print("💡 This prevents idle timeout during long scraping sessions")

In [ ]:
# Settings
SAVE_DIR = '/content/drive/MyDrive/dai-keiba/data/raw'
BASIC_CSV = 'database_basic.csv'      # Input: Basic data
DETAILS_CSV = 'database_details.csv'  # Output: Details data

In [ ]:
# RaceScraper Class (Horse History + Pedigree)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
import re
from datetime import datetime
import time
import os
import gc
from tqdm.auto import tqdm

class RaceScraper:
    def __init__(self):
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
        }
    
    def _get_soup(self, url):
        try:
            time.sleep(1)
            response = requests.get(url, headers=self.headers, timeout=10)
            response.encoding = response.apparent_encoding
            if response.status_code == 200:
                return BeautifulSoup(response.text, 'html.parser')
        except Exception as e:
            print(f"Error fetching {url}: {e}")
        return None
    
    def get_horse_profile(self, horse_id):
        """血統情報を取得"""
        url = f"https://db.netkeiba.com/horse/ped/{horse_id}/"
        soup = self._get_soup(url)
        data = {"father": "", "mother": "", "bms": ""}
        if not soup: return data
        
        try:
            table = soup.select_one("table.blood_table")
            if table:
                rows = table.find_all("tr")
                if len(rows) >= 17:
                    r0 = rows[0].find_all("td")
                    if r0:
                        data["father"] = r0[0].text.strip().split('\n')[0].strip()
                    r16 = rows[16].find_all("td")
                    if len(r16) >= 2:
                        data["mother"] = r16[0].text.strip().split('\n')[0].strip()
                        data["bms"] = r16[1].text.strip().split('\n')[0].strip()
        except Exception as e:
            pass
        return data
    
    def extract_run_style(self, passing_str):
        if not isinstance(passing_str, str): return 3
        try:
            cleaned = re.sub(r'[^0-9-]', '', passing_str)
            parts = [int(p) for p in cleaned.split('-') if p]
            if not parts: return 3
            first_corner = parts[0]
            if first_corner == 1: return 1
            elif first_corner <= 4: return 2
            elif first_corner <= 9: return 3
            else: return 4
        except: return 3
    
    def get_past_races(self, horse_id, current_race_date, n_samples=5):
        """過去レース履歴を取得"""
        url = f"https://db.netkeiba.com/horse/result/{horse_id}/"
        soup = self._get_soup(url)
        if not soup: return pd.DataFrame()
        
        table = soup.select_one("table.db_h_race_results")
        if not table:
            tables = soup.find_all("table")
            for t in tables:
                if "着順" in t.text:
                    table = t
                    break
        if not table: return pd.DataFrame()
        
        try:
            df = pd.read_html(io.StringIO(str(table)))[0]
            df = df.dropna(how='all')
            df.columns = df.columns.astype(str).str.replace(r'\s+', '', regex=True)
            
            if '日付' in df.columns:
                df['date_obj'] = pd.to_datetime(df['日付'], format='%Y/%m/%d', errors='coerce')
                df = df.dropna(subset=['date_obj'])
                current_date = pd.to_datetime(current_race_date)
                df = df[df['date_obj'] < current_date]
                df = df.sort_values('date_obj', ascending=False)
            
            if df.empty: return df
            df = df.head(n_samples)
            
            if '通過' in df.columns:
                df['run_style_val'] = df['通過'].apply(self.extract_run_style)
            else:
                df['run_style_val'] = 3
            
            column_map = {
                '日付': 'date', '開催': 'venue', '天気': 'weather', 'レース名': 'race_name',
                '着順': 'rank', '枠番': 'waku', '馬番': 'umaban', '騎手': 'jockey',
                '斤量': 'weight_carried', '馬場': 'condition', 'タイム': 'time',
                '着差': 'margin', '上り': 'last_3f', '通過': 'passing', '馬体重': 'horse_weight',
                'run_style_val': 'run_style', '単勝': 'odds', 'オッズ': 'odds', '距離': 'raw_distance'
            }
            df.rename(columns=column_map, inplace=True)
            
            if 'raw_distance' in df.columns:
                def parse_dist(x):
                    if not isinstance(x, str): return None, None
                    surf = None; dist = None
                    if '芝' in x: surf = '芝'
                    elif 'ダ' in x: surf = 'ダ'
                    elif '障' in x: surf = '障'
                    match = re.search(r'(\d+)', x)
                    if match: dist = int(match.group(1))
                    return surf, dist
                
                parsed = df['raw_distance'].apply(parse_dist)
                df['course_type'] = parsed.apply(lambda x: x[0])
                df['distance'] = parsed.apply(lambda x: x[1])
            else:
                df['course_type'] = None
                df['distance'] = None
            
            return df
        except Exception as e:
            return pd.DataFrame()

print("✅ RaceScraper loaded")

In [ ]:
# Main Execution: Fetch Details for each horse
def run_details_scraping():
    basic_path = os.path.join(SAVE_DIR, BASIC_CSV)
    details_path = os.path.join(SAVE_DIR, DETAILS_CSV)
    
    # Load basic data
    if not os.path.exists(basic_path):
        print(f"❌ Basic CSVが見つかりません: {basic_path}")
        print("   先にColab_JRA_Basic.ipynbを実行してください")
        return
    
    print("📖 Basic CSVを読み込み中...")
    basic_df = pd.read_csv(basic_path, dtype=str)
    print(f"   {len(basic_df)} 行読み込み")
    
    # Extract unique horses
    horse_race_pairs = basic_df[['race_id', 'horse_id', '日付']].drop_duplicates()
    horse_race_pairs = horse_race_pairs[horse_race_pairs['horse_id'].notna()]
    horse_race_pairs = horse_race_pairs[horse_race_pairs['horse_id'] != '']
    
    print(f"📋 ユニーク馬: {len(horse_race_pairs)} 件")
    
    # Check existing
    existing_pairs = set()
    if os.path.exists(details_path):
        existing_df = pd.read_csv(details_path, dtype=str, usecols=['race_id', 'horse_id'])
        existing_pairs = set(zip(existing_df['race_id'], existing_df['horse_id']))
        print(f"💾 既存取得済み: {len(existing_pairs)} 件")
    
    # Calculate diff
    target_pairs = []
    for _, row in horse_race_pairs.iterrows():
        pair = (row['race_id'], row['horse_id'])
        if pair not in existing_pairs:
            target_pairs.append(row)
    
    print(f"🚀 今回取得対象: {len(target_pairs)} 件")
    
    if not target_pairs:
        print("✅ 全て取得済みです。")
        return
    
    # Scrape
    scraper = RaceScraper()
    buffer = []
    chunk_size = 50
    
    for i, row in enumerate(tqdm(target_pairs)):
        race_id = row['race_id']
        horse_id = row['horse_id']
        race_date = row['日付']
        
        try:
            # Pedigree
            pedigree = scraper.get_horse_profile(horse_id)
            
            # History
            history_df = scraper.get_past_races(horse_id, race_date, n_samples=5)
            
            # Build row
            row_data = {
                'race_id': race_id,
                'horse_id': horse_id,
                'father': pedigree['father'],
                'mother': pedigree['mother'],
                'bms': pedigree['bms']
            }
            
            # Add past races
            for j in range(5):
                prefix = f"past_{j+1}"
                if j < len(history_df):
                    r = history_df.iloc[j]
                    row_data[f"{prefix}_date"] = r.get('date', '')
                    row_data[f"{prefix}_rank"] = r.get('rank', '')
                    row_data[f"{prefix}_time"] = r.get('time', '')
                    row_data[f"{prefix}_run_style"] = r.get('run_style', '')
                    row_data[f"{prefix}_race_name"] = r.get('race_name', '')
                    row_data[f"{prefix}_last_3f"] = r.get('last_3f', '')
                    row_data[f"{prefix}_horse_weight"] = r.get('horse_weight', '')
                    row_data[f"{prefix}_jockey"] = r.get('jockey', '')
                    row_data[f"{prefix}_condition"] = r.get('condition', '')
                    row_data[f"{prefix}_odds"] = r.get('odds', '')
                    row_data[f"{prefix}_weather"] = r.get('weather', '')
                    row_data[f"{prefix}_distance"] = r.get('distance', '')
                    row_data[f"{prefix}_course_type"] = r.get('course_type', '')
                else:
                    for col in ['date','rank','time','run_style','race_name','last_3f',
                               'horse_weight','jockey','condition','odds','weather','distance','course_type']:
                        row_data[f"{prefix}_{col}"] = ''
            
            buffer.append(row_data)
            
        except Exception as e:
            print(f"  Error: {e}")
        
        # Save chunk
        if len(buffer) >= chunk_size or (i == len(target_pairs) - 1 and buffer):
            df_chunk = pd.DataFrame(buffer)
            
            if not os.path.exists(details_path):
                df_chunk.to_csv(details_path, index=False)
            else:
                df_chunk.to_csv(details_path, mode='a', header=False, index=False)
            
            print(f"  Saved {len(buffer)} horses")
            buffer = []
            gc.collect()
    
    print("✅ 詳細情報取得完了")
    print(f"\n次のステップ: merge_jra_data.py を実行してdatabase.csvを作成")

print("✅ Main function ready")

In [ ]:
# Run
run_details_scraping()